In [ ]:
from common.np import *  # import numpy as np (or import cupy as np)
from common.layers import *
from common.functions import softmax, sigmoid

In [ ]:
class RNN:
    def __init__(self,Wx,Wh,b):
        self.params=[Wx,Wh,b]
        self.grads=[np.zeros_like(Wx),np.zeros_like(Wh),np.zeros_like(b)]
        self.cache=None

    def forward(self,x,h_prev):
        Wx,Wh,b=self.params
        t=np.dot(h_prev,Wh)+np.dot(x,Wx)+b
        h_next=np.tanh(t)
        self.cache=(x,h_prev,h_next)

        return h_next

    def backward(self,dh_next):
        Wx,Wh,b=self.params
        x,h_prev,h_next=self.cache
        dt=dh_next*(1.-h_next**2)
        db=np.sum(dt,axis=0)    #repeatノードは集約（N×H→H）
        dWh=np.dot(h_prev.T,dt) #逆伝搬してきた方(dt)はいじらないで行列の形状が正しくなるように掛ける方を転置する
        dh_prev=np.dot(dt,Wh.T)
        dWx=np.dot(x.T,dt)
        dx=np.dot(dt,Wx.T)

        self.grads[0][...]=dWx
        self.grads[1][...]=dWh
        self.grads[2][...]=db

        return dx,dh_prev

In [ ]:
class TimeRNN:
    def __init__(self,Wx,Wh,b,stateful=False):
        self.params=[Wx,Wh,b]
        self.grads=[np.zeros_like(Wx),np.zeros_like(Wh),np.zeros_like(b)]
        self.layers=None    #複数のRNNレイヤを格納するメンバ変数を初期化
        self.h,self.dh=None,None    #hとdhを初期化
        self.stateful=stateful  #hを保持するかどうかを管理する引数をメンバ変数に保持

    def set_state(self,h):
        self.h=h

    def reset_state(self):
        self.h=None

    def forward(self,xs):
        Wx,Wh,b=self.params
        N,T,D=xs.shape  #Tは１ブロックに含まれるRNNレイヤの数（=時系列データの数）
        D,H=Wx.shape

        self.layers=[]  #複数のRNNレイヤを格納するリストを作成
        hs=np.empty((N,T,H),dtype='f')

        if not self.stateful or self.h is None: #statefulがFalseまたはself.hの初回呼び出し時は、self.hをゼロ行列で初期化する
            self.h=np.zeros((N,H),dtype='f')

        for t in range(T):
            layer=RNN(*self.params)
            self.h=layer.forward(xs[:,t,:],self.h)  #xsにおける各バッチのt番目のベクトルをforwardの引数として渡す
            hs[:,t,:]=self.h
            self.layers.append(layer)

        return hs

    def backward(self,dhs):
        Wx,Wh,b=self.params
        N,T,H=dhs.shape
        D,H=Wx.shape
        dxs=np.empty((N,T,D),dtype='f')
        dh=0
        grads=[0,0,0]   #[dWx,dWh,db]

        for t in reversed(range(T)):
            layer=self.layers[t]
            dx,dh=layer.backward(dhs[:,t,:]+dh) #順伝搬の際に分岐（コピー）したhは逆伝搬では合算されて伝搬する
            dxs[:,t,:]=dx

            for i,grad in enumerate(layer.grads):  #各RNN layerの勾配（dWx,dWh,db）同士を足し合わせる
                grads[i]+=grad

        for i,grad in enumerate(grads):
            self.grads[i][...]=grad
        self.dh=dh

        return dxs

In [ ]:
class TimeEmbedding:
    def __init__(self,W):
        self.params=[W]
        self.grads=[np.zeros_like(W)]
        self.layers=None
        self.W=W

    def forward(self,xs):
        N,T=xs.shape    #N:バッチサイズ,T:時系列データの数
        V,D=self.W.shape    #V:vocab_size,D:単語の分散表現の次元数
        self.layers=[]
        out=np.empty((N,T,D),dtype='f')
        for t in range(T):
            layer=Embedding(self.W)
            out[:,t,:]=layer.forward(xs[:,t])
            self.layers.append(layer)

        return out

    def backward(self,dout):
        N,T,D=dout.shape
        grad=0
        for t in reversed(range(T)):
            layer=self.layers[t]
            layer.backward(dout[:,t,:])
            grad+=layer.grads[0]    #すべてのRNNレイヤの勾配を加算する
        self.grads[0][...]=grad

        return None   

In [ ]:
class Simple_TimeAffine:
    def __init__(self,W,b):
        self.params=[W,b]
        self.grads=[np.zeros_like(W),np.zeros_like(b)]
        self.W=W
        self.b=b
        self.layers=None

    def forward(self,xs):
        N,T,D=xs.shape
        D,M=self.W.shape
        self.layers=[]
        out=np.empty((N,T,M),dtype='f')

        for t in range(T):
            layer=Affine(self.W,self.b)
            out[:,t,:]=layer.forward(xs[:,t,:])
            self.layers.append(layer)

        return out

    def backward(self,dout):
        N,T,M=dout.shape
        dW,db=0,0
        dxs=np.empty((N,T,D),dtype='f')
        for t in reversed(range(T)):
            layer=self.layers[t]
            dxs[:,t,:]=layer.backward(dout[:,t,:])
            dW+=layer.grads[0]
            db+=layer.grads[1]
        
        self.grads[0][...]=dW
        self.grads[1][...]=db

        return dxs

In [ ]:
#Simple_TimeAffineの高速版
class TimeAffine:
    def __init__(self,W,b):
        self.params=[W,b]
        self.grads=[np.zeros_like(W),np.zeros_like(b)]
        self.W=W
        self.b=b
        self.x=None

    def forward(self,x):
        N,T,D=x.shape
        D,M=self.W.shape
        rx=x.reshape(N*T,-1)
        out=np.dot(rx,self.W)+self.b
        self.x=x

        return out.reshape(N,T,-1)

    def backward(self,dout):
        N,T,M=dout.shape
        dout=dout.reshape(N*T,-1)
        rx=self.x.reshape(N*T,-1)
        db=np.sum(dout,axis=0)
        dW=np.dot(rx.T,dout)
        dx=np.dot(dout,self.W.T)
        dx=dx.reshape(N,T,-1)

        self.grads[0][...]=dW
        self.grads[1][...]=db

        return dx

In [ ]:
class Simple_TimeSoftmaxWithLoss:
    def __init__(self):
        self.params,self.grads=[],[]
        self.cache=None

    def forward(self,xs,ts):
        N,T,V=xs.shape
        layers=[]
        loss=0
        
        for t in range(T):
            layer=SoftmaxWithLoss()
            loss+=layer.forward(xs[:,t,:],ts[:,t])  #損失の合計を計算
            layers.append(layer)
        loss/=T #時系列データの平均を取る(バッチの平均はSoftmaxWithLossレイヤで行われている)
        self.cache=(layers,xs)

        return loss

    def backward(self,dout=1):
        layers,xs=self.cache
        N,T,V=xs.shape
        dxs=np.empty((N,T,V),dtype='f')
        dout*=1/T

        for t in range(T):
            layer=layers[t]
            dxs[:,t,:]=layer.backward(dout)

        return dxs

In [ ]:
#Simple_TimeSoftmaxWithLossの高速版
class TimeSoftmaxWithLoss:
    def __init__(self):
        self.params,self.grads=[],[]
        self.ignor_label=-1
        self.cache=None

    def forward(self,xs,ts):
        N,T,V=xs.shape

        if ts.ndim==3:  #one-hotの場合、ラベル表記に変換する
            ts=ts.argmax(axis=2)

        mask=(ts!=self.ignor_label) #正解ラベルがignor_labelと等しくないときTrue

        #バッチ分と時系列分をまとめる
        xs=xs.reshape(N*T,-1)
        ts=ts.reshape(N*T)
        mask=mask.reshape(N*T)

        ys=softmax(xs)
        ls=np.log(ys[np.arange(N*T),ts])    #ysの正解ラベルに該当するスコアの対数を取る
        ls*=mask    #ignor_labelに該当するデータは損失を0にする
        loss=-np.sum(ls)    #損失の合計を計算
        loss/=mask.sum()  #maskのTrueの数(N*Tの内のignor_label以外のデータ数)で割り、平均を取る
        self.cache=(ts,ys,mask,(N,T,V))

        return loss

    def backward(self,dout):
        ts,ys,mask,(N,T,V)=self.cache
        dx=ys
        dx[np.arange(N*T),ts]-=1  #softmaxWithLossの逆伝搬はy-t,tは正解ラベル(=1)
        dx*=dout
        dx/=mask.sum()
        dx *= mask[:, np.newaxis]  # ignore_labelに該当するデータは勾配を0にする
        dx = dx.reshape((N, T, V))

        return dx

In [ ]:
class LSTM:
    def __init__(self,Wx,Wh,b):
        self.params=[Wx,Wh,b]   #f,g,i,o４つ分の重みがひとつの重みにまとめられている　Wx.shape=(D,4H),Wh.shape=(H,4H),b.shape=(4H,)
        self.grads=[np.zeros_like(Wx),np.zeros_like(Wh),np.zeros_like(b)]
        self.cache=None

    def forward(self,x,h_prev,c_prev):
        Wx,Wh,b=self.params
        H=Wh.shape[0]

        A=np.dot(x,Wx)+np.dot(h_prev,Wh)+b  #まとめてAffine変換 A.shapeは(N,4H)
        #対応する要素に分割
        f=A[:,:H]
        g=A[:,H:2*H]
        i=A[:,2*H:3*H]
        o=A[:,3*H:]

        f=sigmoid(f)
        g=np.tanh(g)
        i=sigmoid(i)
        o=sigmoid(o)

        c_next=f*c_prev+g*i #アダマール積(要素ごとの積)
        h_next=o*np.tanh(c_next)    #アダマール積（要素ごとの積）

        self.cache=(x,h_prev,c_prev,i,f,g,o,c_next)

        return h_next,c_next

    def backward(self,dh_next,dc_next):
        Wx,Wh,b=self.params
        x,h_prev,c_prev,i,f,g,o,c_next=self.cache

        tanh_c_next=np.tanh(c_next)
        dtanh_c_next=dh_next*o  #アダマール積の逆伝搬は入力をひっくり返してアダマール積を取る
        ds=dc_next+dtanh_c_next*(1-tanh_c_next**2)  #tanhの逆伝搬は1-y**2
        dc_prev=ds*f    #アダマール積の逆伝搬は入力をひっくり返してアダマール積を取る

        do=dh_next*tanh_c_next
        dA_o=do*o*(1-o) #sigmoidの逆伝搬はy(1-y)
        di=ds*g
        dA_i=di*i*(1-i) #sigmoidの逆伝搬はy(1-y)
        dg=ds*i
        dA_g=dg*(1-g**2)    ##tanhの逆伝搬は1-y**2
        df=ds*c_prev
        dA_f=df*f*(1-f) #sigmoidの逆伝搬はy(1-y)

        dA=np.hstack((dA_f,dA_g,dA_i,dA_o))   #分割した要素を統合する→dA.shapeは(N,4H)
        db=np.sum(dA,axis=0)    #db.shapeは(4H,)
        dWh=np.dot(h_prev.T,dA)    #dWh.shapeは(H,4H)
        dh_prev=np.dot(dA,Wh.T) #dh_prev.shapeは(N,H)
        dWx=np.dot(x.T,dA)  #dWx.shapeは(D,4H)
        dx=np.dot(dA,Wx.T)  #dx.shapeは(N,D)

        self.grads[0][...]=dWx
        self.grads[1][...]=dWh
        self.grads[2][...]=db

        return dx,dh_prev,dc_prev

In [ ]:
class TimeLSTM:
    def __init__(self,Wx,Wh,b,stateful=False):
        self.params=[Wx,Wh,b]
        self.grads=[np.zeros_like(Wx),np.zeros_like(Wh),np.zeros_like(b)]
        self.layers=None
        self.h,self.c=None,None
        self.dh=None
        self.stateful=stateful

    def forward(self,xs):
        Wx,Wh,b=self.params
        N,T,D=xs.shape
        H=Wh.shape[0]
        self.layers=[]

        if not self.stateful or self.h is None:
            self.h=np.zeros((N,H)).astype('f')
        if not self.stateful or self.c is None:
            self.c=np.zeros((N,H)).astype('f')

        hs=np.empty((N,T,H)).astype('f')

        for t in range(T):
            layer=LSTM(Wx,Wh,b)
            self.h,self.c=layer.forward(xs[:,t,:],self.h,self.c)
            hs[:,t,:]=self.h
            self.layers.append(layer)

        return hs

    def backward(self,dhs):
        Wx,Wh,b=self.params
        N,T,H=dhs.shape
        D=Wx.shape[0]

        dxs=np.empty((N,T,D)).astype('f')
        dh,dc=0,0

        grads=[0,0,0]
        for t in reversed(range(T)):
            layer=self.layers[t]
            dx,dh,dc=layer.backward(dhs[:,t,:]+dh,dc)
            dxs[:,t,:]=dx
            for i,grad in enumerate(layer.grads):
                grads[i]+=grad

        for i,grad in enumerate(grads):
            self.grads[i][...]=grad

        self.dh=dh

        return dxs

    def set_state(self,h,c=None):
        self.h,self.c=h,c

    def reset_state(self):
        self.h,self.c=None,None

In [ ]:
class TimeDropout:
    def __init__(self, dropout_ratio=0.5):
        self.params, self.grads = [], []
        self.dropout_ratio = dropout_ratio
        self.mask = None
        self.train_flg = True

    def forward(self, xs):
        if self.train_flg:
            flg = np.random.rand(*xs.shape) > self.dropout_ratio    #0から1までの乱数を一様分布から生成し、ドロップアウト率と比較した結果をブール値で格納
            #ドロップアウト率だけ出力が小さくなるため、スケールをもとに戻す
            #（dropout_ratio=1/5の場合、1/5のニューロンが無視されるから出力は4/5になる→5/4を掛けて出力を1に戻す）
            scale = 1 / (1.0 - self.dropout_ratio)
            self.mask = flg.astype(np.float32) * scale  #flgのうちTrueの箇所はTrue(=1)*scale=scale,Falseの箇所はFalse(=0)*scale=0としたものをmaskとする

            return xs * self.mask
        else:
            return xs

    def backward(self, dout):
        return dout * self.mask #順伝搬時に無視したニューロンでは逆伝搬もストップする(伝搬する値を0にする)